In [6]:
import json
import numpy as np

['/Users/matt/ML_experimentation', '/Users/matt/miniconda3/envs/tf_metal/lib/python311.zip', '/Users/matt/miniconda3/envs/tf_metal/lib/python3.11', '/Users/matt/miniconda3/envs/tf_metal/lib/python3.11/lib-dynload', '', '/Users/matt/.local/lib/python3.11/site-packages', '/Users/matt/miniconda3/envs/tf_metal/lib/python3.11/site-packages']


In [41]:
f = open("data/StarTrek_scripts/all_scripts_raw.json")
json_file = json.load(f)
f.close()
#start with TOS: might be more manageable
TOS_scripts=json_file['TOS']
TOS_scripts['episode 12'][:1000]

'\n\n\n\n\n\nThe Star Trek Transcripts - The Menagerie, Part 2\n\n\n\nThe\r\nMenagerie, part 2\nStardate:3012.4 \r\nOriginal Airdate: 24 Nov, 1966\n\n\n\n\n\n\n \r\nPersonal log, stardate 3013.1. I find it hard to believe the events of\r\nthe past twenty four hours or the plea of Mister Spock\r\nstanding general court-martial.\r\n\n[Recap of part one] \nMENDEZ: How do you plead to the charge of\r\nunlawfully taking command?\r\nSPOCK: Guilty.\r\nMENDEZ: Of sabotaging the computers of this vessel and locking it on a\r\ncourse for planet Talos Four?\r\nSPOCK: Guilty.\r\nMENDEZ: And of forcibly attempting to transport Captain Pike to that\r\nplanet?\r\nSPOCK: Guilty.\r\nKIRK [OC]: Why? Why does Spock want to take to that forbidden world his\r\nformer\r\ncaptain. Mutilated by a recent space disaster,\r\nnow a shell of a man, unable to speak or move? The only answer Spock\r\nwould give was on the hearing-room screen.\r\nHow Spock could do this he refused to explain, but there before our\r\ne

In [42]:
# for key, value in TOS_scripts.items() :
#     print(key)
print(TOS_scripts['episode 12'][:1000])

#Input: sequence of 128 tokens. Output







The Star Trek Transcripts - The Menagerie, Part 2



The
Menagerie, part 2
Stardate:3012.4 
Original Airdate: 24 Nov, 1966






 
Personal log, stardate 3013.1. I find it hard to believe the events of
the past twenty four hours or the plea of Mister Spock
standing general court-martial.

[Recap of part one] 
MENDEZ: How do you plead to the charge of
unlawfully taking command?
SPOCK: Guilty.
MENDEZ: Of sabotaging the computers of this vessel and locking it on a
course for planet Talos Four?
SPOCK: Guilty.
MENDEZ: And of forcibly attempting to transport Captain Pike to that
planet?
SPOCK: Guilty.
KIRK [OC]: Why? Why does Spock want to take to that forbidden world his
former
captain. Mutilated by a recent space disaster,
now a shell of a man, unable to speak or move? The only answer Spock
would give was on the hearing-room screen.
How Spock could do this he refused to explain, but there before our
eyes, actual images from thirteen years ago of Captain Pike as h


### General plan

I want to generate a star trek script. The model will be some kind of transformer. The input is a series of tokens, I'll start with 128 tokens (needs padded in case the input is shorter). The output is the next word, i.e. input: [The, quick, brown], output: [quick, brown, fox]

To make this, that means I need to:
- parse the scripts, remove line breaks and things.

    - Also need to remove episode title at beginning and copyright stuff at the end.
    - Probably should add a character for stage direction, or perhaps if I keep the colons such that kirk: is distinct from kirk.
    
- Create segments of input tokens in batches of 128

- Embed the wordings

- Split train/test data

- Create model and train
    


In [103]:
import re

def add_special_tokens(script):
    # Replace character names
    script = re.sub(r'\n([A-Z ]+):', r' <CHAR> \1:', script) #adds <CHAR> token any time theres a new line followed by "<CAPITALLETTERS>:"
    # You can add more substitutions here for stage directions or other special tokens
    script = re.sub(r'\[([^\]]+)\]', r' <LOC> \1 <LOC>', script) #add <LOC> token to indicate location
    script = re.sub(r'\(([^)]+)\)', r' <SD> \1 <SD>', script)
    return script

def remove_metadata(script):
    # Find the position of the 17th newline character
    start_pos = -1
    for _ in range(17):
        start_pos = script.find('\n', start_pos + 1)
        
    # Slice the string from the character after the 8th newline
    if start_pos != -1:
        script = script[start_pos + 1:]
        
    return script

for i in range(10):
    # print(TOS_scripts['episode '+str(i)][:1100])
    print(remove_metadata(add_special_tokens(TOS_scripts['episode '+str(i)][:1100])))
    




  <LOC> Bridge <LOC>
 <CHAR> SPOCK: It can't be the screen then. Definitely something out there,
 <CHAR> TYLER: It's coming at the speed of light, collision course. The. 
 <CHAR> GARISON: It's a radio wave, sir. We're passing through an old-style
 <CHAR> PIKE: They were keyed to cause interference and attract attention this
 <CHAR> GARISON: A ship in trouble making a forced landing, sir. That's it. No
 <CHAR> SPOCK: Their call letters check with a survey expedition. SS Columbia.
 <CHAR> TYLER: It would take that long for a radio beam to travel from there to
here.
SPOC




  Captain's log, Stardate
1513.1. Our position, orbiting planet M-113. On board the Enterprise,
Mister Spock temporarily in command. On the planet the ruins of an
ancient and long-dead civilisation. Ship's surgeon McCoy and myself are
now beaming down to the planet's surface. Our mission, routine medical
examination of archaeologist Robert Crater and his wife Nancy.
Routine but for the fact that Nancy Crater is tha